In [13]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [14]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('evaluate#.csv')
# df = pd.read_csv('evaluate_seg.csv')

df = df.drop(columns=['pbid'])
df = df[df['ARG'] <= 100000]
# df = df[df['classcial'] <= 500]
# df = df[df['in_constraints_probs'] >= 99]
grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    'iteration_count':'mean',
    'classcial':'mean',
    'run_times':'mean',
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
})

values=["ARG", 'best_solution_probs', 'classcial', 'run_times','iteration_count']

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid', 'variables','constraints'], columns='method', values=values)
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['ChocoSolver', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver']
method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver', 'QtoSolver ', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver', 'QtoSimplifyDiscardSegmentedSolver']
# method_order = ['ChocoSolver', 'QtoSolver', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver']
# method_order = ['ChocoSolver']
# method_order = ['QtoSimplifyDiscardSegmentedSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([values, method_order]))

pivot_df

ARG                                       \
                              HeaSolver PenaltySolver ChocoSolver QtoSolver    
pkid variables constraints                                                     
0    6         3              47.009022     51.434867    0.095949        NaN   
1    15        8             157.863780    119.204157    0.294871        NaN   
2    21        12            162.335417    119.435028    0.360775        NaN   
3    28        15            274.899127    194.480496    0.571255        NaN   
4    12        6             177.985535    132.972738    0.215230        NaN   
5    15        9             302.760310    226.202840    0.608376        NaN   
6    20        8             521.207211    343.638284    1.100469        NaN   
7    24        12            796.905222    611.960307    1.530496        NaN   
8    8         6             354.850046    346.276477    0.319811        NaN   
9    15        8             416.912298    371.135567    0.945742        NaN   
10   18        9             621.042581    493.135243    1.391617        NaN   
11   21        10           1120.940823   1184.159265    3.501929        NaN   
12   7         4              65.804441     74.048259    0.063660        NaN   
13   10        5             194.259825    161.300335    0.422540        NaN   
14   14        6             193.995703    169.861083    0.538621        NaN   
15   18        7             403.311300    323.687016    0.744384        NaN   
16   9         4             943.826595    836.484974    3.936367        NaN   
17   12        5            1100.215629    958.824591    7.268241        NaN   
18   16        6            1423.962180   1256.712019   22.096429        NaN   
19   20        7            1615.093999   1512.362341   26.214772        NaN   

                                                                       \
                           QtoSimplifySolver QtoSimplifyDiscardSolver   
pkid variables constraints                                              
0    6         3                    0.011123                 0.008403   
1    15        8                    0.133791                 0.115298   
2    21        12                   0.199832                 0.138274   
3    28        15                   0.653707                 0.526405   
4    12        6                    0.180615                 0.146504   
5    15        9                    0.228525                 0.433982   
6    20        8                    0.389243                 0.457047   
7    24        12                   0.759606                 0.933495   
8    8         6                    0.033675                 0.066786   
9    15        8                    0.860032                 0.887477   
10   18        9                    1.221257                 1.211682   
11   21        10                   2.926642                 3.163162   
12   7         4                    0.009018                 0.005512   
13   10        5                    0.287797                 0.281008   
14   14        6                    0.372690                 0.346194   
15   18        7                    0.637039                 0.551791   
16   9         4                    0.108716                 0.241201   
17   12        5                    0.617482                 0.695985   
18   16        6                    6.721675                 4.024189   
19   20        7                   21.695573                16.518567   

                                                              \
                           QtoSimplifyDiscardSegmentedSolver   
pkid variables constraints                                     
0    6         3                                    0.001673   
1    15        8                                    0.141654   
2    21        12                                   0.348680   
3    28        15                                   0.546348   
4    12        6                                    0.220680   
5  

In [2]:
a = pivot_df[('classcial', 'ChocoSolver')].sum()
b = pivot_df[('classcial', 'QtoSimplifyDiscardSolver')].sum()
c = pivot_df[('classcial', 'ChocoSolver')].mean()
d = pivot_df[('classcial', 'QtoSimplifyDiscardSolver')].mean()
print(f'choco:{c}, qtodiscard:{d}')
print(d/c)


choco:13.735534870049596, qtodiscard:54.865855254791335
3.994446213698355


In [3]:
# # 假设“variables”和“constraints”列作为多重索引列的一部分
# # 使用 .loc 选择多重索引列中含“variables”和“constraints”相关的列
# selected_columns = pivot_df.loc[:, pivot_df.columns.get_level_values(0).isin(['variables', 'constraints'])]

# selected_columns


In [4]:
# 选取 'depth' 下的所有列，并剔除 'QtoSolver ' 和 'QtoSimplifySolver'
df_1 = pivot_df['ARG'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 选取 'culled_depth_per_param' 下的 'QtoSimplifyDiscardSolver' 列
df_2 = pivot_df['classcial'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 合并处理后的 'depth' 列与 'QtoSimplifyDiscardSolver' 列，并指定后缀避免冲突
culled_depth_df = df_1.join(df_2, lsuffix='_1', rsuffix='_2')

# 按行遍历输出值，每个值用 & 分隔
for row in culled_depth_df.itertuples(index=False):
    formatted_row = [f"{round(x, 3):6.03f}" for x in row]
    print(" & ".join(formatted_row))


 0.096 &  0.008 &  1.029 &  0.407
 0.295 &  0.115 &  5.404 & 12.889
 0.361 &  0.138 &  9.936 & 36.205
 0.571 &  0.526 & 26.186 & 165.529
 0.215 &  0.147 &  3.589 &  1.207
 0.608 &  0.434 &  2.728 &  0.816
 1.100 &  0.457 & 21.431 & 16.321
 1.530 &  0.933 & 14.413 & 11.091
 0.320 &  0.067 &  1.967 &  0.633
 0.946 &  0.887 &  5.403 &  1.801
 1.392 &  1.212 & 13.625 & 46.346
 3.502 &  3.163 & 19.589 & 90.572
 0.064 &  0.006 &  1.379 &  0.462
 0.423 &  0.281 &  2.591 &  1.166
 0.539 &  0.346 &  6.691 & 15.763
 0.744 &  0.552 & 10.564 & 26.028
 3.936 &  0.241 &  3.377 &  3.293
 7.268 &  0.696 & 10.002 & 16.528
22.096 &  4.024 & 37.482 & 105.628
26.215 & 16.519 & 77.327 & 544.632


In [5]:
# # 从 pivot_df 中提取 variables 和 constraints 列
# variables_constraints_df = pivot_df.index.to_frame(index=False)[['variables', 'constraints']]
# variables_constraints_df

In [6]:
# 提取所需列
best_solution_probs_Qto_discard = pivot_df['best_solution_probs']['QtoSimplifyDiscardSolver']
best_solution_probs_choco = pivot_df['best_solution_probs']['ChocoSolver']
arg_Qto_discard = pivot_df['ARG']['QtoSimplifyDiscardSolver']
arg_choco = pivot_df['ARG']['ChocoSolver']

# 方法1：每行相除取平均
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的倍数
row_wise_ratio_best_solution_probs = (best_solution_probs_Qto_discard / best_solution_probs_choco).mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的倍数
row_wise_ratio_arg = (arg_choco / arg_Qto_discard).mean()

# 方法2：整列取平均后相除
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的整列平均的倍数
col_avg_ratio_best_solution_probs = best_solution_probs_Qto_discard.mean() / best_solution_probs_choco.mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的整列平均的倍数
col_avg_ratio_arg = arg_choco.mean() / arg_Qto_discard.mean()

# 输出结果
print(f"行平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {row_wise_ratio_best_solution_probs:.2f}")
print(f"行平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {row_wise_ratio_arg:.2f}")
print(f"列平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {col_avg_ratio_best_solution_probs:.2f}")
print(f"列平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {col_avg_ratio_arg:.2f}")


行平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): 1.63
行平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): 4.12
列平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): 1.35
列平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): 2.35


In [7]:

# # 读入 CSV 文件
# df = pd.read_csv('evaluate_TSP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [8]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_SCP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [9]:
# 定义格式化函数
import numpy as np
def format_value(val):
    if np.isnan(val):  # 如果是NaN值则返回空字符
        return "NaN"
    elif val < 100:  # 如果值小于100，保留3位有效数字
        return f"{val:.2f}"
    else:  # 如果值大于等于100，不保留小数点
        return f"{val:.0f}"

# 按行遍历输出，每个值用 & 分隔
for row in pivot_df.itertuples(index=False):
    formatted_row = [format_value(x) for x in row]
    print(" & ".join(formatted_row))

0.10 & 0.01 & 0.01 & 88.34 & 96.52 & 96.69 & 1.03 & 0.69 & 0.41 & 119 & 136 & 69.52 & 108 & 90.97 & 40.73
0.29 & 0.13 & 0.12 & 37.72 & 71.39 & 67.70 & 5.40 & 22.20 & 12.89 & 121 & 453 & 463 & 110 & 301 & 271
0.36 & 0.20 & 0.14 & 23.23 & 51.90 & 60.34 & 9.94 & 46.01 & 36.20 & 124 & 453 & 546 & 112 & 301 & 301
0.57 & 0.65 & 0.53 & 15.75 & 13.14 & 21.55 & 26.19 & 189 & 166 & 121 & 453 & 655 & 110 & 301 & 301
0.22 & 0.18 & 0.15 & 54.85 & 55.90 & 64.12 & 3.59 & 5.50 & 1.21 & 118 & 432 & 150 & 107 & 288 & 87.46
0.61 & 0.23 & 0.43 & 34.54 & 64.09 & 38.98 & 2.73 & 6.13 & 0.82 & 114 & 453 & 113 & 103 & 300 & 64.00
1.10 & 0.39 & 0.46 & 10.24 & 39.12 & 32.19 & 21.43 & 31.05 & 16.32 & 116 & 453 & 491 & 105 & 301 & 282
1.53 & 0.76 & 0.93 & 5.78 & 32.79 & 24.22 & 14.41 & 40.09 & 11.09 & 118 & 453 & 346 & 107 & 301 & 206
0.32 & 0.03 & 0.07 & 85.88 & 96.83 & 95.50 & 1.97 & 1.20 & 0.63 & 117 & 144 & 88.54 & 105 & 97.24 & 54.82
0.95 & 0.86 & 0.89 & 30.57 & 33.04 & 30.91 & 5.40 & 13.26 & 1.80 & 119 & 453

In [10]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_old.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [11]:
pivot_df['ARG']

,,,ChocoSolver,QtoSimplifySolver,QtoSimplifyDiscardSolver
pkid,variables,constraints,,,
0,6,3,0.095949,0.011123,0.008403
1,15,8,0.294871,0.133791,0.115298
2,21,12,0.360775,0.199832,0.138274
3,28,15,0.571255,0.653707,0.526405
4,12,6,0.215230,0.180615,0.146504
5,15,9,0.608376,0.228525,0.433982
6,20,8,1.100469,0.389243,0.457047
7,24,12,1.530496,0.759606,0.933495
8,8,6,0.319811,0.033675,0.066786


In [12]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eval_eliminate copy 2.csv')

# df = df.drop(columns=['pbid','layers', 'variables', 'constraints'])

# gr    ouped_df = df.groupby(['pkid',  'method'], as_index=False).agg({
#     "successrate": 'mean',
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# # grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# # grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df